In [1]:
import csv
from konlpy.tag import Kkma
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

kkma = Kkma()

In [2]:
movie_title={1:"겟아웃",2:"부산행",3:"서치",4:"해피데스데이",5:"곡성"}
for i in range(len(movie_title)):
    rtitle = "%s_최종본.csv" %(movie_title[i+1])
    uptitle= "%s_상위tf.csv" %(movie_title[i+1])
    # Get out, Train to Busan, Search over 3.58, The Wailing over 3.81, Happy Deathday over 3.32이상
    wtitle = "%s_연관성.csv" %(movie_title[i+1])
    
    csv_file = open(rtitle, 'r')
    doc = csv.reader(csv_file)
    next(doc) # skip header
    
    # Split by sentence
    def split_sentence(doc):
        text = doc[2]
        sentence=kkma.sentences(text)
        return sentence
    # Save by sentence
    result = []
    for line in doc:
        result.extend(split_sentence(line))
    
    ## Top tf Preprocess2
    f=open(uptitle,'r')
    rf=csv.reader(f)
    next(rf)
    TF=[]
    for row in rf:
        TF.append(row[0])

    stop=['이', '을', '스', '여','노', '은', '아', '터', '드', '마', '자', '고', '레','거','개방','관람','씬','내용','결국','그런데',
          '거','늘', '의', '라', '누', '어', '건','진','자', '마','로','스','까','은','시','드','곽','왜','그리하','약간','갑자기',
          '마','저','거','라','까','분','늘','자','의','어','스','여','을','몇','몇몇','걸','시','드','단','서','인','블록버스터',
          '늘','메','파','자','김','티','리','바','스','건','대','새','어','보','시','디','르','을','고','예고편','스포일러','장르',
          '어쩌','듯하','경','어느','결말','이때','재미있','그리하','이번','조금',
          '스','자','누','터','라','뜨','다','막','분','임','위','하다','리','사','유','면','감독','스포','줄거리','영화']
    newTF=[]
    for word in TF:
        if word not in stop:
            newTF.append(word)

    # Analyze results divided by sentence by morpheme
    def split_morphs(doc):
        doc_morphs = kkma.morphs(doc)
        return doc_morphs
    
    dataset2=[]
    for i in range(len(result)):
        a=split_morphs(result[i])
        dataset= []
        for w in a:
            if w in newTF: 
                dataset.append(w)
        dataset2.append(dataset)
    
    # Check which top tf word each sentence has
    dataset3=[]
    for k in range(len(dataset2)):
        if dataset2[k] != []:
            dataset3.append(dataset2[k])
    
    # Association Analyze 
    te = TransactionEncoder()
    te_ary = te.fit(dataset3).transform(dataset3)
    df = pd.DataFrame(te_ary, columns=te.columns_) #for better understanding chage into Dataframe 
    
    frequent_itemsets3 = apriori(df, min_support=0.003, max_len=3, use_colnames=True)
    frequent_itemsets2 = apriori(df, min_support=0.003, max_len=2, use_colnames=True)
    frequentThree=frequent_itemsets3[len(frequent_itemsets2):len(frequent_itemsets3)]    
    
    frequentThree.to_csv(wtitle, encoding = 'cp949')       
    print(wtitle,"저장 완료")

겟아웃_연관성.csv 저장 완료
부산행_연관성.csv 저장 완료
서치_연관성.csv 저장 완료
해피데스데이_연관성.csv 저장 완료
곡성_연관성.csv 저장 완료
